In [0]:
# Gold Layer Configuration
storage_account = "datamigrationsathya"
container = "datalake"
source_system = "mysql"

# Paths
silver_base = f"abfss://{container}@{storage_account}.dfs.core.windows.net/silver/"
gold_base = f"abfss://{container}@{storage_account}.dfs.core.windows.net/gold/"

print("🏆 Gold Layer - Business Aggregations")
print("=" * 70)
print(f"Silver source: {silver_base}")
print(f"Gold target: {gold_base}")
print("=" * 70)

In [0]:
# Load and explore key Silver tables
retail_customers = spark.read.format("delta").load(f"{silver_base}{source_system}/retail_db/customer_details/")
retail_orders = spark.read.format("delta").load(f"{silver_base}{source_system}/retail_db/orders/")
retail_users = spark.read.format("delta").load(f"{silver_base}{source_system}/retail_db/users/")
customer_orders = spark.read.format("delta").load(f"{silver_base}{source_system}/retail_db/customer_orders/")

print("Silver Layer Tables Loaded:")
print(f"  • customer_details: {retail_customers.count()} records")
print(f"  • orders: {retail_orders.count()} records")
print(f"  • users: {retail_users.count()} records")
print(f"  • customer_orders: {customer_orders.count()} records")

print("\nSample schemas:")
print("\ncustomer_details:")
retail_customers.printSchema()
print("\norders:")
retail_orders.printSchema()

In [0]:
# Business Aggregation 1: Customer 360 - Complete customer profile with order metrics
from pyspark.sql.functions import *

customer_360 = retail_customers.alias('c') \
    .join(customer_orders.alias('co'), col('c.customer_id') == col('co.customer_id'), 'left') \
    .groupBy(
        col('c.customer_id'),
        col('c.customer_name'),
        col('c.customer_email')
    ).agg(
        count('co.order_id').alias('total_orders'),
        coalesce(sum('co.order_amount'), lit(0)).alias('total_revenue'),
        coalesce(avg('co.order_amount'), lit(0)).alias('avg_order_value'),
        max('co.order_date').alias('last_order_date'),
        min('co.order_date').alias('first_order_date'),
        datediff(max('co.order_date'), min('co.order_date')).alias('customer_lifetime_days'),
        current_timestamp().alias('created_at')
    ) \
    .withColumn('customer_segment', 
        when(col('total_orders') >= 10, 'VIP')
        .when(col('total_orders') >= 5, 'Loyal')
        .when(col('total_orders') >= 1, 'Active')
        .otherwise('Inactive')
    )

# Write to Gold layer
gold_customer_360_path = f"{gold_base}business/customer_360/"
customer_360.write.format("delta").mode("overwrite").save(gold_customer_360_path)

print("✅ Customer 360 View created")
print(f"   Path: {gold_customer_360_path}")
print(f"   Records: {customer_360.count()}")
display(customer_360.orderBy(col('total_revenue').desc()).limit(10))

In [0]:
# Business Aggregation 2: Daily Sales Metrics
daily_sales = customer_orders \
    .withColumn('order_date_only', to_date('order_date')) \
    .groupBy('order_date_only').agg(
        count('order_id').alias('total_orders'),
        sum('order_amount').alias('total_revenue'),
        avg('order_amount').alias('avg_order_value'),
        countDistinct('customer_id').alias('unique_customers'),
        min('order_amount').alias('min_order'),
        max('order_amount').alias('max_order'),
        current_timestamp().alias('created_at')
    ) \
    .withColumn('revenue_per_customer', col('total_revenue') / col('unique_customers')) \
    .orderBy('order_date_only')

gold_daily_sales_path = f"{gold_base}business/daily_sales_metrics/"
daily_sales.write.format("delta").mode("overwrite").save(gold_daily_sales_path)

print("✅ Daily Sales Metrics created")
print(f"   Path: {gold_daily_sales_path}")
print(f"   Records: {daily_sales.count()}")
display(daily_sales.orderBy(col('order_date_only').desc()).limit(10))

In [0]:
# Business Aggregation 3: Product Performance (if product info exists in orders)
if 'product_id' in retail_orders.columns or 'product_name' in retail_orders.columns:
    product_cols = ['product_id'] if 'product_id' in retail_orders.columns else ['product_name']
    
    product_performance = retail_orders \
        .groupBy(*product_cols).agg(
            count('order_id').alias('total_orders'),
            sum('order_amount').alias('total_revenue'),
            avg('order_amount').alias('avg_price'),
            countDistinct('user_id').alias('unique_buyers'),
            current_timestamp().alias('created_at')
        ) \
        .withColumn('revenue_rank', dense_rank().over(Window.orderBy(col('total_revenue').desc())))
    
    gold_product_path = f"{gold_base}business/product_performance/"
    product_performance.write.format("delta").mode("overwrite").save(gold_product_path)
    
    print("✅ Product Performance created")
    print(f"   Path: {gold_product_path}")
    display(product_performance.orderBy('revenue_rank').limit(10))
else:
    print("⚠️  Product columns not found in orders table - skipping product performance")

In [0]:
# Business Aggregation 4: User Activity Summary
user_activity = retail_users.alias('u') \
    .join(retail_orders.alias('o'), col('u.user_id') == col('o.user_id'), 'left') \
    .groupBy(
        col('u.user_id'),
        col('u.country') if 'country' in retail_users.columns else lit('Unknown').alias('country'),
        col('u.state') if 'state' in retail_users.columns else lit('Unknown').alias('state')
    ).agg(
        count('o.order_id').alias('total_orders'),
        coalesce(sum('o.amount'), lit(0)).alias('total_spent'),
        max('o.order_date').alias('last_activity_date'),
        current_timestamp().alias('created_at')
    ) \
    .withColumn('user_status',
        when(col('total_orders') == 0, 'Never Ordered')
        .when(datediff(current_date(), col('last_activity_date')) > 90, 'Inactive')
        .when(datediff(current_date(), col('last_activity_date')) > 30, 'At Risk')
        .otherwise('Active')
    )

gold_user_activity_path = f"{gold_base}business/user_activity_summary/"
user_activity.write.format("delta").mode("overwrite").save(gold_user_activity_path)

print("✅ User Activity Summary created")
print(f"   Path: {gold_user_activity_path}")
print(f"   Records: {user_activity.count()}")
display(user_activity.groupBy('user_status').count().orderBy('count', ascending=False))

In [0]:
# Business Aggregation 5: Geographic Sales Analysis
if 'country' in retail_users.columns:
    geo_sales = retail_users.alias('u') \
        .join(retail_orders.alias('o'), col('u.user_id') == col('o.user_id'), 'inner') \
        .groupBy(
            col('u.country'),
            col('u.state') if 'state' in retail_users.columns else lit('Unknown').alias('state')
        ).agg(
            count('o.order_id').alias('total_orders'),
            sum('o.order_amount').alias('total_revenue'),
            avg('o.order_amount').alias('avg_order_value'),
            countDistinct('u.user_id').alias('unique_customers'),
            current_timestamp().alias('created_at')
        ) \
        .withColumn('revenue_per_customer', col('total_revenue') / col('unique_customers'))
    
    gold_geo_sales_path = f"{gold_base}business/geographic_sales/"
    geo_sales.write.format("delta").mode("overwrite").save(gold_geo_sales_path)
    
    print("✅ Geographic Sales Analysis created")
    print(f"   Path: {gold_geo_sales_path}")
    display(geo_sales.orderBy(col('total_revenue').desc()).limit(10))
else:
    print("⚠️  Geographic columns not found - skipping geographic analysis")

In [0]:
# Business Aggregation 6: Employee & Department Analytics (students_db)
employees = spark.read.format("delta").load(f"{silver_base}{source_system}/students_db/employees/")
departments = spark.read.format("delta").load(f"{silver_base}{source_system}/students_db/departments/")

dept_analytics = employees.alias('e') \
    .join(departments.alias('d'), col('e.department_id') == col('d.department_id'), 'left') \
    .groupBy(
        col('d.department_id'),
        col('d.department_name') if 'department_name' in departments.columns else lit('Unknown').alias('department_name')
    ).agg(
        count('e.employee_id').alias('total_employees'),
        avg('e.salary').alias('avg_salary') if 'salary' in employees.columns else lit(0).alias('avg_salary'),
        min('e.salary').alias('min_salary') if 'salary' in employees.columns else lit(0).alias('min_salary'),
        max('e.salary').alias('max_salary') if 'salary' in employees.columns else lit(0).alias('max_salary'),
        sum('e.salary').alias('total_payroll') if 'salary' in employees.columns else lit(0).alias('total_payroll'),
        current_timestamp().alias('created_at')
    )

gold_dept_path = f"{gold_base}business/department_analytics/"
dept_analytics.write.format("delta").mode("overwrite").save(gold_dept_path)

print("✅ Department Analytics created")
print(f"   Path: {gold_dept_path}")
print(f"   Records: {dept_analytics.count()}")
display(dept_analytics.orderBy(col('total_employees').desc()))

In [0]:
# Business Aggregation 7: Monthly Sales Trends
monthly_trends = customer_orders \
    .withColumn('year', year('order_date')) \
    .withColumn('month', month('order_date')) \
    .withColumn('year_month', date_format('order_date', 'yyyy-MM')) \
    .groupBy('year', 'month', 'year_month').agg(
        count('order_id').alias('total_orders'),
        sum('order_amount').alias('total_revenue'),
        avg('order_amount').alias('avg_order_value'),
        countDistinct('customer_id').alias('unique_customers'),
        current_timestamp().alias('created_at')
    ) \
    .withColumn('revenue_growth_pct',
        ((col('total_revenue') - lag('total_revenue').over(Window.orderBy('year', 'month'))) / 
         lag('total_revenue').over(Window.orderBy('year', 'month')) * 100)
    ) \
    .orderBy('year', 'month')

gold_monthly_path = f"{gold_base}business/monthly_trends/"
monthly_trends.write.format("delta").mode("overwrite").save(gold_monthly_path)

print("✅ Monthly Trends created")
print(f"   Path: {gold_monthly_path}")
print(f"   Records: {monthly_trends.count()}")
display(monthly_trends.orderBy(col('year').desc(), col('month').desc()).limit(12))

## 🏆 Gold Layer - Business Aggregations Complete

### Created Analytics Tables:

1. **Customer 360 View** - Complete customer profiles with order metrics and segmentation
   - Metrics: Total orders, revenue, AOV, lifetime value, customer segments (VIP/Loyal/Active/Inactive)

2. **Daily Sales Metrics** - Day-over-day sales performance
   - Metrics: Orders, revenue, AOV, unique customers, revenue per customer

3. **Product Performance** - Product-level sales analysis
   - Metrics: Orders, revenue, average price, unique buyers, revenue ranking

4. **User Activity Summary** - User engagement and status tracking
   - Metrics: Order count, total spent, last activity, user status (Active/At Risk/Inactive)

5. **Geographic Sales Analysis** - Location-based sales insights
   - Metrics: Orders, revenue, AOV by country/state, revenue per customer

6. **Department Analytics** - Employee and department metrics
   - Metrics: Employee count, salary statistics, total payroll by department

7. **Monthly Trends** - Time-series sales analysis
   - Metrics: Monthly orders, revenue, growth rates, customer trends

### Usage:
- All tables stored in Delta format at: `gold/business/`
- Ready for BI dashboards, reporting, and analytics
- Includes metadata timestamps for tracking freshness

In [0]:
# Configuration for Silver layer access
storage_account = "datamigrationsathya"
container = "datalake"
source_system = "mysql"

# Silver layer base path
silver_base_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/silver/"

print("Discovering Silver layer tables...")
print("=" * 70)

# Discover databases and tables in Silver layer
all_silver_tables = []

try:
    # Check MySQL tables
    mysql_path = f"{silver_base_path}{source_system}/"
    databases = dbutils.fs.ls(mysql_path)
    
    for db in databases:
        if db.isDir():
            db_name = db.name.rstrip('/')
            tables = dbutils.fs.ls(db.path)
            for table in tables:
                if table.isDir():
                    table_name = table.name.rstrip('/')
                    all_silver_tables.append({
                        'source': 'mysql',
                        'database': db_name,
                        'table': table_name,
                        'path': table.path
                    })
                    print(f"  ✓ {db_name}.{table_name}")
    
    # Check Event Hub tables
    eventhub_path = f"{silver_base_path}eventhub/"
    try:
        eh_tables = dbutils.fs.ls(eventhub_path)
        for table in eh_tables:
            if table.isDir():
                table_name = table.name.rstrip('/')
                all_silver_tables.append({
                    'source': 'eventhub',
                    'database': 'eventhub',
                    'table': table_name,
                    'path': table.path
                })
                print(f"  ✓ eventhub.{table_name}")
    except:
        pass
    
    print("=" * 70)
    print(f"Total Silver tables found: {len(all_silver_tables)}")
    
except Exception as e:
    print(f"Error: {str(e)}")


In [0]:
# Sample key tables from Silver layer
tables_to_sample = [
    ('retail_db', 'customer_details'),
    ('retail_db', 'orders'),
    ('retail_db', 'users'),
    ('retail_db', 'customer_orders'),
    ('students_db', 'employees'),
    ('students_db', 'departments')
]

for db, table in tables_to_sample:
    path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/silver/{source_system}/{db}/{table}/"
    df = spark.read.format("delta").load(path)
    print(f"\n{'='*70}")
    print(f"{db}.{table}")
    print(f"{'='*70}")
    print(f"Schema: {', '.join([f'{f.name}({f.dataType.simpleString()})' for f in df.schema.fields[:8]])}")
    print(f"Records: {df.count()}")
    df.limit(3).show(truncate=False)


In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window

# Configuration
storage_account = "datamigrationsathya"
container = "datalake"
source_system = "mysql"
silver_base = f"abfss://{container}@{storage_account}.dfs.core.windows.net/silver/"
gold_base = f"abfss://{container}@{storage_account}.dfs.core.windows.net/gold/"

print("🏆 Creating Gold Layer Business Aggregations...\n")

# Load Silver tables
retail_customers = spark.read.format("delta").load(f"{silver_base}{source_system}/retail_db/customer_details/")
retail_orders = spark.read.format("delta").load(f"{silver_base}{source_system}/retail_db/orders/")
retail_users = spark.read.format("delta").load(f"{silver_base}{source_system}/retail_db/users/")
customer_orders = spark.read.format("delta").load(f"{silver_base}{source_system}/retail_db/customer_orders/")
employees = spark.read.format("delta").load(f"{silver_base}{source_system}/students_db/employees/")
departments = spark.read.format("delta").load(f"{silver_base}{source_system}/students_db/departments/")

gold_tables_created = []

# 1. Customer 360 View
customer_360 = retail_customers.alias('c') \
    .join(customer_orders.alias('co'), col('c.customer_id') == col('co.customer_id'), 'left') \
    .groupBy(col('c.customer_id'), col('c.customer_name'), col('c.customer_email')).agg(
        count('co.order_id').alias('total_orders'),
        coalesce(sum('co.order_amount'), lit(0)).alias('total_revenue'),
        coalesce(avg('co.order_amount'), lit(0)).alias('avg_order_value'),
        max('co.order_date').alias('last_order_date'),
        current_timestamp().alias('created_at')
    ).withColumn('customer_segment', 
        when(col('total_orders') >= 10, 'VIP')
        .when(col('total_orders') >= 5, 'Loyal')
        .when(col('total_orders') >= 1, 'Active')
        .otherwise('Inactive')
    )
path = f"{gold_base}business/customer_360/"
customer_360.write.format("delta").mode("overwrite").save(path)
gold_tables_created.append(('customer_360', customer_360.count(), path))

# 2. Daily Sales Metrics
daily_sales = customer_orders \
    .withColumn('order_date_only', to_date('order_date')) \
    .groupBy('order_date_only').agg(
        count('order_id').alias('total_orders'),
        sum('order_amount').alias('total_revenue'),
        avg('order_amount').alias('avg_order_value'),
        countDistinct('customer_id').alias('unique_customers'),
        current_timestamp().alias('created_at')
    ).orderBy('order_date_only')
path = f"{gold_base}business/daily_sales_metrics/"
daily_sales.write.format("delta").mode("overwrite").save(path)
gold_tables_created.append(('daily_sales_metrics', daily_sales.count(), path))

# 3. User Activity Summary
user_activity = retail_users.alias('u') \
    .join(retail_orders.alias('o'), col('u.user_id') == col('o.user_id'), 'left') \
    .groupBy(col('u.user_id'), col('u.country')).agg(
        count('o.order_id').alias('total_orders'),
        coalesce(sum('o.amount'), lit(0)).alias('total_spent'),
        max('o.order_date').alias('last_activity_date'),
        current_timestamp().alias('created_at')
    ).withColumn('user_status',
        when(col('total_orders') == 0, 'Never Ordered')
        .when(datediff(current_date(), col('last_activity_date')) > 90, 'Inactive')
        .otherwise('Active')
    )
path = f"{gold_base}business/user_activity_summary/"
user_activity.write.format("delta").mode("overwrite").save(path)
gold_tables_created.append(('user_activity_summary', user_activity.count(), path))

# 4. Geographic Sales Analysis
geo_sales = retail_users.alias('u') \
    .join(retail_orders.alias('o'), col('u.user_id') == col('o.user_id'), 'inner') \
    .groupBy(col('u.country')).agg(
        count('o.order_id').alias('total_orders'),
        sum('o.amount').alias('total_revenue'),
        avg('o.amount').alias('avg_order_value'),
        countDistinct('u.user_id').alias('unique_customers'),
        current_timestamp().alias('created_at')
    ).withColumn('revenue_per_customer', col('total_revenue') / col('unique_customers'))
path = f"{gold_base}business/geographic_sales/"
geo_sales.write.format("delta").mode("overwrite").save(path)
gold_tables_created.append(('geographic_sales', geo_sales.count(), path))

# 5. Department Analytics
dept_analytics = employees.alias('e') \
    .join(departments.alias('d'), col('e.department_id') == col('d.department_id'), 'left') \
    .groupBy(col('d.department_id'), col('d.department_name')).agg(
        count('e.employee_id').alias('total_employees'),
        current_timestamp().alias('created_at')
    )
path = f"{gold_base}business/department_analytics/"
dept_analytics.write.format("delta").mode("overwrite").save(path)
gold_tables_created.append(('department_analytics', dept_analytics.count(), path))

# 6. Monthly Trends
monthly_trends = customer_orders \
    .withColumn('year', year('order_date')) \
    .withColumn('month', month('order_date')) \
    .withColumn('year_month', date_format('order_date', 'yyyy-MM')) \
    .groupBy('year', 'month', 'year_month').agg(
        count('order_id').alias('total_orders'),
        sum('order_amount').alias('total_revenue'),
        avg('order_amount').alias('avg_order_value'),
        countDistinct('customer_id').alias('unique_customers'),
        current_timestamp().alias('created_at')
    ).orderBy('year', 'month')
path = f"{gold_base}business/monthly_trends/"
monthly_trends.write.format("delta").mode("overwrite").save(path)
gold_tables_created.append(('monthly_trends', monthly_trends.count(), path))

# Summary
print("\n" + "="*70)
print("✅ Gold Layer Created Successfully!")
print("="*70)
for table_name, count, path in gold_tables_created:
    print(f"  ✓ {table_name}: {count} records")
print(f"\nLocation: {gold_base}business/")
print(f"Total tables: {len(gold_tables_created)}")

In [0]:
# Verify the gold/business/ directory exists and list all tables
import os

print("Verifying Gold Layer...\n")

try:
    gold_business_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/gold/business/"
    tables = dbutils.fs.ls(gold_business_path)
    
    print(f"📁 {gold_business_path}")
    print("="*70)
    
    for table in tables:
        if table.isDir():
            table_name = table.name.rstrip('/')
            # Read and count records
            df = spark.read.format("delta").load(table.path)
            record_count = df.count()
            print(f"  ✓ {table_name}: {record_count} records")
    
    print("\n✅ Gold layer verified successfully!")
    
except Exception as e:
    print(f"❌ Error: {str(e)}")
    print("\nThe gold/business/ directory may not exist yet. Run the previous cell first.")